In [0]:
from google.colab import drive
drive.mount("/content/drive")

In [0]:
from __future__ import print_function, division

import torch 
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import time
import os
import copy 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

plt.ion()

import sklearn.svm
from sklearn.metrics import accuracy_score
use_gpu = torch.cuda.is_available()

if use_gpu:
    print("CUDA kullanılıyor")

In [0]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
     'test': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_transformsInception = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(299),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
     'test': transforms.Compose([
        transforms.Resize(299),
        transforms.CenterCrop(299),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

data_dir = '/content/drive/My Drive/datasetnoisy'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
image_datasetInception = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transformsInception[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train','val', 'test']}
dataloaderInception =  {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=16,
                                             shuffle=True, num_workers=4)
              for x in ['train','val', 'test']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val','test']}
class_names = image_datasets['train'].classes
for x in ["train", "val", "test"]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
print(image_datasets["train"].classes)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [0]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    #plt.figure(figsize=(10, 10))
    plt.axis('off')
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)
def show_databatch(inputs, classes):
    out = torchvision.utils.make_grid(inputs)
    imshow(out, title = [class_names[x] for x in classes])
inputs, classes = next(iter(dataloaders["train"]))
show_databatch(inputs,classes)

In [0]:
def visualize_model(model, num_images=6):
    was_training = model.training
    model.eval()
    images_so_far = 0
    fig = plt.figure()

    with torch.no_grad():
        for i, (inputs, labels) in enumerate(dataloaders['test']):
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            for j in range(inputs.size()[0]):
                images_so_far += 1
                ax = plt.subplot(num_images//2, 2, images_so_far)
                ax.axis('off')
                ax.set_title("predicted: {} actual: {}".format(class_names[preds[j]],class_names[labels.data.cpu().numpy()[j]]))
                imshow(inputs.cpu().data[j])

                if images_so_far == num_images:
                    model.train(mode=was_training)
                    return
        model.train(mode=was_training)

In [0]:
def eval_model(vgg, criterion,dtldr):
    since = time.time()
    avg_loss = 0
    avg_acc = 0
    loss_test = 0
    acc_test = 0
    test_batches = len(dtldr["test"])
    print("Hesaplama modeli")
    print('-' * 10)
    for i, data in enumerate(dtldr["test"]):
        print("\rTest batch {}/{}".format(i, test_batches), end = '', flush=True)
        
        vgg.train(False)
        vgg.eval()
        inputs, labels = data
        
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = vgg(inputs)
        
        _, preds = torch.max(outputs.data.cpu(), 1)
        loss = criterion(outputs, labels)       
        loss_test += loss.data
        acc_test += torch.sum(preds.cpu() == labels.data.cpu())
        
        del inputs, labels, outputs, preds
        torch.cuda.empty_cache()
        
    
    avg_loss = loss_test / dataset_sizes["test"]
    avg_acc = acc_test.item() / dataset_sizes["test"]
    
    elapsed_time = time.time() - since
    print()
    print("Hesaplama {:.0f}m {:.0f}s zamanda tamamlandı".format(elapsed_time // 60, elapsed_time % 60))
    print("Ortalama kayıp (test): {:.4f}".format(avg_loss))
    print("Ortalama acc (test): {:.4f}".format(avg_acc))
    print('-' * 10)

In [0]:
def train_model(vgg, criterion, optimizer, scheduler, num_epochs,dtldr):
    since = time.time()
    best_model_wts = copy.deepcopy(vgg.state_dict())
    best_acc = 0.0
    
    avg_loss = 0
    avg_acc = 0
    avg_loss_val = 0
    avg_acc_val = 0
    loss_graph_train = []
    loss_graph_valid = []
    
    train_batches = len(dtldr["train"])
    val_batches = len(dtldr["val"])
    
    for epoch in range(num_epochs):
        print("Epoch {}/{}".format(epoch, num_epochs))
        print('-' * 10)
        
        loss_train = 0
        loss_val = 0
        acc_train = 0
        acc_val = 0
        
        vgg.train(True)
        
        for i, data in enumerate(dtldr["train"]):
            if i % 100 == 0:
                print("\rTraining batch {}/{}".format(i, train_batches), end='', flush=True)
                
            inputs, labels = data
            
            inputs = inputs.to(device)
            labels = labels.to(device)
                
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss.backward()
            optimizer.step()
            
            loss_train += loss.data
            acc_train += torch.sum(preds == labels.data).data.cpu().numpy()
            
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        print()
        # * 2 as we only used half of the dataset
        avg_loss = loss_train  / dataset_sizes["train"]
        avg_acc = acc_train  / dataset_sizes["train"]
        loss_graph_train.append(avg_loss.data.cpu().item())
        vgg.train(False)
        vgg.eval()
            
        for i, data in enumerate(dtldr["val"]):
            if i % 100 == 0:
                print("\rValidation batch {}/{}".format(i, val_batches), end='', flush=True)
                
            inputs, labels = data

            inputs = inputs.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            
            outputs = vgg(inputs)
            
            _, preds = torch.max(outputs.data, 1)
            loss = criterion(outputs, labels)
            
            loss_val += loss.data
            acc_val += torch.sum(preds == labels.data).data.cpu().numpy()
            del inputs, labels, outputs, preds
            torch.cuda.empty_cache()
        
        avg_loss_val = loss_val / dataset_sizes["val"]
        avg_acc_val = acc_val / dataset_sizes["val"]
        
        loss_graph_valid.append(avg_loss_val.data.cpu().item())
        
        print()
        print("Epoch {} result: ".format(epoch))
        print("Avg loss (train): {:.4f}".format(avg_loss))
        print("Avg acc (train): {:.4f}".format(avg_acc))
        print("Avg loss (val): {:.4f}".format(avg_loss_val))
        print("Avg acc (val): {:.4f}".format(avg_acc_val))
        print('-' * 10)
        print()
        
        if avg_acc_val > best_acc:
            best_acc = avg_acc_val
            best_model_wts = copy.deepcopy(vgg.state_dict())
        
    elapsed_time = time.time() - since
    print()
    print("Training completed in {:.0f}m {:.0f}s".format(elapsed_time // 60, elapsed_time % 60))
    print("Best acc: {:.4f}".format(best_acc))
    plt.figure(figsize=(8, 6))
    epochs = []
    for i in range(num_epochs):
      epochs.append(i+1)
    plot1, = plt.plot(np.array(epochs),np.array(loss_graph_train))
    plot2, = plt.plot(np.array(epochs),np.array(loss_graph_valid))
    plt.legend([plot1, plot2], ["train_loss", "validation_loss"])
    plt.xlabel('Epoch')
    plt.ylabel('Average Negative Log Likelihood')
    plt.title('Training and Validation Losses')
    plt.show()
    vgg.load_state_dict(best_model_wts)
    return vgg

In [0]:
resnet18 = models.resnet18(pretrained=True)
alexnet = models.alexnet(pretrained=True)
vgg16 = models.vgg16(pretrained=True)
densenet = models.densenet121(pretrained=True)
inception = models.inception_v3(pretrained=True)

In [0]:
resnet18.fc = nn.Linear(512, len(class_names))
alexnet.classifier[6] = nn.Linear(4096,len(class_names))
num_features = vgg16.classifier[6].in_features
features = list(vgg16.classifier.children())[:-1] 
features.extend([nn.Linear(num_features, len(class_names))])
vgg16.classifier = nn.Sequential(*features)
num_ftrs = densenet.classifier.in_features
densenet.classifier = nn.Linear(num_ftrs, len(class_names))
inception.AuxLogits.fc = nn.Linear(768,len(class_names))
inception.fc = nn.Linear(2048,len(class_names))

In [0]:
resnet18 = resnet18.to(device)
alexnet = alexnet.to(device)
vgg16 = vgg16.to(device)
densenet = densenet.to(device)
inception = inception.to(device)

In [0]:
print(resnet18)

In [0]:
print("VGG16")
criterion = nn.CrossEntropyLoss()
optimizer_ft = optim.SGD(vgg16.parameters(), lr = 0.0001)
# optimizerSGD4 = optim.SGD(vgg16.parameters(), lr = 0.00001)
# optimizerAdam3 = optim.Adam(vgg16.parameters(), lr = 0.0001)
# optimizerSGD3 = optim.SGD(vgg16.parameters(), lr = 0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
# exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
vgg16 = train_model(vgg16, criterion, optimizer_ft, exp_lr_scheduler, 20,dataloaders)
torch.save(vgg16.state_dict(), 'VGG16_FineTune.pt')
eval_model(vgg16, criterion,dataloaders)

print("ResNet18")

optimizer_ft = optim.SGD(resnet18.parameters(), lr = 0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
resnet18 = train_model(resnet18,criterion,optimizer_ft,exp_lr_scheduler,20,dataloaders)
torch.save(resnet18.state_dict(), 'ResNet18_FineTune.pt')
eval_model(resnet18, criterion,dataloaders)

print("AlexNet")

optimizer_ft = optim.SGD(alexnet.parameters(), lr = 0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
alexnet = train_model(alexnet, criterion, optimizer_ft, exp_lr_scheduler, 20,dataloaders)
torch.save(alexnet.state_dict(), 'AlexNet_FineTune.pt')
eval_model(alexnet, criterion,dataloaders)


print("DenseNet")
optimizer_ft = optim.SGD(densenet.parameters(), lr = 0.0001)
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size = 4, gamma = 0.1)
densenet = train_model(densenet, criterion, optimizer_ft, exp_lr_scheduler, 20,dataloaders)
torch.save(densenet.state_dict(), 'denseNet_FineTune.pt')
eval_model(densenet, criterion , dataloaders)

In [0]:
visualize_model(alexnet,4)